<a href="https://colab.research.google.com/github/j-min/HiREST/blob/main/custom_video_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# HiREST inference demo on a custom video with a joint baseline model

* Project page: https://hirest-cvpr2023.github.io
* Code: https://github.com/j-min/HiREST

In [1]:
# Make sure GPU is used
!nvidia-smi

Sat Sep 14 15:40:03 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.183.01             Driver Version: 535.183.01   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce GTX 1650        Off | 00000000:01:00.0  On |                  N/A |
| N/A   45C    P3              14W /  50W |    544MiB /  4096MiB |     35%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
!git clone https://github.com/j-min/HiREST

Cloning into 'HiREST'...
remote: Enumerating objects: 122, done.
remote: Counting objects: 100% (30/30), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 122 (delta 12), reused 12 (delta 6), pack-reused 92
Receiving objects: 100% (122/122), 3.72 MiB | 24.91 MiB/s, done.
Resolving deltas: 100% (21/21), done.


In [ ]:
cd HiREST

# Installation

First install required libraries and download all pretrained weights

In [2]:
!pip install -r requirements.txt

  Cloning https://github.com/openai/whisper.git (to revision eff383b27b783e280c089475852ba83f20f64998) to /tmp/pip-req-build-gu74m10b
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-gu74m10b
  Running command git rev-parse -q --verify 'sha^eff383b27b783e280c089475852ba83f20f64998'
  Running command git fetch -q https://github.com/openai/whisper.git eff383b27b783e280c089475852ba83f20f64998
  Running command git checkout -q eff383b27b783e280c089475852ba83f20f64998
  Resolved https://github.com/openai/whisper.git to commit eff383b27b783e280c089475852ba83f20f64998
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/bckim92/language-evaluation.git to /tmp/pip-req-build-dv0xkfc6
  Running command git clone --filter=blob:none --quiet https://github.com/bckim92/language-evaluation.git /tmp/pip-req-build-dv0xkfc6
  Resolved https://github.com/bckim92/language-evaluation.git to commit 8e9e03feb7f5e605f20fc0c8dba0e2

# Download Checkpoints

In [3]:
!mkdir ./checkpoints/
!mkdir ./checkpoints/hirest_joint_model/
!mkdir ./pretrained_weights/

In [4]:
# Pretrained CLIP4Caption
!wget https://huggingface.co/j-min/HiREST-baseline/resolve/main/clip4caption_vit-b-32_model.bin
!mv clip4caption_vit-b-32_model.bin ./pretrained_weights/clip4caption_vit-b-32_model.bin

# EVA-CLIP-G (frozen visual/textual encoder)
!wget https://huggingface.co/BAAI/EVA/resolve/main/eva_clip_psz14.pt
!mv eva_clip_psz14.pt ./pretrained_weights/eva_clip_psz14.pt

# Finetuned HiREST joint baseline model
!wget https://huggingface.co/j-min/HiREST-baseline/resolve/main/HiREST_BEST.pth
!mv HiREST_BEST.pth ./checkpoints/hirest_joint_model/BEST.pth

--2024-09-14 15:53:34--  https://huggingface.co/j-min/HiREST-baseline/resolve/main/clip4caption_vit-b-32_model.bin
Resolving huggingface.co (huggingface.co)... 18.67.216.43, 18.67.216.58, 18.67.216.51, ...
Connecting to huggingface.co (huggingface.co)|18.67.216.43|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/43/e5/43e5477f8142d4d657272c2d83709064f52cea70061d7f463fe4e9973cc4707b/e4814e953cb088c2f096ac346b003a750bed4f899b7af7bbd9513d43247837be?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27clip4caption_vit-b-32_model.bin%3B+filename%3D%22clip4caption_vit-b-32_model.bin%22%3B&response-content-type=application%2Foctet-stream&Expires=1726563214&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcyNjU2MzIxNH19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9yZXBvcy80My9lNS80M2U1NDc3ZjgxNDJkNGQ2NTcyNzJjMmQ4MzcwOTA2NGY1MmNlYTcwMDYxZDdmNDYzZmU0ZTk5NzNjYzQ3MDdiL2U

# Create Data Dirs

In [ ]:
!mkdir custom_video_pipeline/
!mkdir custom_video_pipeline/data
!mkdir custom_video_pipeline/data/eva_clip_features/
!mkdir custom_video_pipeline/data/ASR_feats_all-MiniLM-L6-v2/
!mkdir custom_video_pipeline/data/ASR/
!mkdir custom_video_pipeline/data/raw_frames/
!mkdir custom_video_pipeline/data/audio/
!mkdir custom_video_pipeline/data/splits/
!mkdir custom_video_pipeline/data/videos/

# Download a sample video

In [ ]:
# !wget https://huggingface.co/j-min/HiREST-baseline/resolve/main/4Gx9W0XFAkA.mp4
# !mv 4Gx9W0XFAkA.mp4 ./custom_video_pipeline/data/videos/4Gx9W0XFAkA.mp4

--2023-03-28 17:30:11--  https://huggingface.co/j-min/HiREST-baseline/resolve/main/4Gx9W0XFAkA.mp4
Resolving huggingface.co (huggingface.co)... 35.173.225.216, 34.203.133.210, 52.2.178.255, ...
Connecting to huggingface.co (huggingface.co)|35.173.225.216|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/43/e5/43e5477f8142d4d657272c2d83709064f52cea70061d7f463fe4e9973cc4707b/89fe3b10d4318445d42460c0614dc258a94707214e5498bc344eafc7d85131b8?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%274Gx9W0XFAkA.mp4%3B+filename%3D%224Gx9W0XFAkA.mp4%22%3B&response-content-type=video%2Fmp4&Expires=1680283813&Policy=eyJTdGF0ZW1lbnQiOlt7IlJlc291cmNlIjoiaHR0cHM6Ly9jZG4tbGZzLmh1Z2dpbmdmYWNlLmNvL3JlcG9zLzQzL2U1LzQzZTU0NzdmODE0MmQ0ZDY1NzI3MmMyZDgzNzA5MDY0ZjUyY2VhNzAwNjFkN2Y0NjNmZTRlOTk3M2NjNDcwN2IvODlmZTNiMTBkNDMxODQ0NWQ0MjQ2MGMwNjE0ZGMyNThhOTQ3MDcyMTRlNTQ5OGJjMzQ0ZWFmYzdkODUxMzFiOD9yZXNwb25zZS1jb250ZW50LWRpc3Bvc2l0aW9uPSomcmVz

# Create Video Data Structure

In [ ]:
import json

In [ ]:
data = {
    "Make Butter Biscuits": {
        "4Gx9W0XFAkA.mp4": {
            "relevant": True,
            "clip": True,
            "v_duration": 299.73,
            "bounds": [ 0, 1 ],
            "steps": [ ]
        }
    }
}

with open("./custom_video_pipeline/data/splits/all_data_test.json", 'w') as f:
    json.dump(data, f)

# Extract ASR Features
*Note: Due to slight randomness with Whisper, ASR results may vary between runs

In [ ]:
!python ./extraction/whisper_ASR/extract_audio.py --workers 1 --data_path ./custom_video_pipeline/data/splits/all_data_test.json --save_path ./custom_video_pipeline/data/audio/ --video_path ./custom_video_pipeline/data/videos/

Extracting audio from videos...
Found 1 videos at custom_video_pipeline/data/videos
Will save audio to custom_video_pipeline/data/audio
Multiprocessing with 1 workers
  0% 0/1 [00:00<?, ?it/s]size=    9367kB time=00:04:59.74 bitrate= 256.0kbits/s speed= 659x    
100% 1/1 [00:01<00:00,  1.27s/it]


In [ ]:
!python ./extraction/whisper_ASR/extract_ASR.py --model small.en --audio_dir ./custom_video_pipeline/data/audio/ --asr_dir ./custom_video_pipeline/data/ASR/

Namespace(model='small.en', audio_dir='./custom_video_pipeline/data/audio/', asr_dir='./custom_video_pipeline/data/ASR/')
100%|███████████████████████████████████████| 461M/461M [00:13<00:00, 36.1MiB/s]
Model loaded:  small.en at device cuda:0
  0% 0/1 [00:00<?, ?it/s]2023-03-28 17:31:21.234631: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-03-28 17:31:21.234751: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-03-28 17:31:21.234773: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvi

In [ ]:
# ASR result
!head -n 20 ./custom_video_pipeline/data/ASR/4Gx9W0XFAkA.srt

1
00:00:00,000 --> 00:00:12,320
What's up YouTube, y'all know I'm good?

2
00:00:12,320 --> 00:00:13,320
You good?

3
00:00:13,320 --> 00:00:14,320
You good?

4
00:00:14,320 --> 00:00:15,320
I'm good, you good?

5
00:00:15,320 --> 00:00:16,320
Okay good.



In [ ]:
!python ./extraction/whisper_ASR/extract_ASR_embedding.py --model sentence-transformers/all-MiniLM-L6-v2 --asr_dir ./custom_video_pipeline/data/ASR/ --save_dir ./custom_video_pipeline/data/ASR_feats_all-MiniLM-L6-v2/

Namespace(model='sentence-transformers/all-MiniLM-L6-v2', asr_dir='./custom_video_pipeline/data/ASR/', save_dir='./custom_video_pipeline/data/ASR_feats_all-MiniLM-L6-v2/')
model loaded:  sentence-transformers/all-MiniLM-L6-v2
ASR_feats_dir:  custom_video_pipeline/data/ASR_feats_all-MiniLM-L6-v2
100% 1/1 [00:00<00:00,  1.50it/s]


# Extract Video Features

In [ ]:
# extract video frames in 1fps
!python ./extraction/video_features/extract_frames.py --workers 1 --data_path ./custom_video_pipeline/data/splits/all_data_test.json --save_path ./custom_video_pipeline/data/raw_frames/ --video_path ./custom_video_pipeline/data/videos/

Namespace(workers=1, data_path='./custom_video_pipeline/data/splits/all_data_test.json', save_path='./custom_video_pipeline/data/raw_frames/', video_path='./custom_video_pipeline/data/videos/')
CV2 version: 4.7.0
Loaded data from ./custom_video_pipeline/data/splits/all_data_test.json
Saving frames to custom_video_pipeline/data/raw_frames
100% 1/1 [00:00<00:00, 15420.24it/s]
Extracting frames from 1 videos
Using 1 worker
100% 1/1 [01:23<00:00, 83.81s/it]
Done


In [ ]:
# extract EVA-CLIP-G features
!python ./extraction/video_features/extract_features.py --frame_path ./custom_video_pipeline/data/raw_frames/ --save_path ./custom_video_pipeline/data/eva_clip_features/ --batch_size 10

Namespace(device=0, slice_start=0, slice_end=-1, frame_path='./custom_video_pipeline/data/raw_frames/', save_path='./custom_video_pipeline/data/eva_clip_features/', batch_size=10)
<All keys matched successfully>
100% 1/1 [01:00<00:00, 60.51s/it]


In [ ]:
!python ./extraction/video_features/check_feature_size.py --data_file ./custom_video_pipeline/data/splits/all_data_test.json --feature_folder ./custom_video_pipeline/data/eva_clip_features/

100% 1/1 [00:00<00:00, 614.82it/s]


# Run the Model End-to-End for all tasks:

moment_retrieval -> moment_segmentation -> step_captioning

In [9]:
!python run.py \
        --data_dir './custom_pipeline/splits/' \
        --video_feature_dir './custom_pipeline/eva_clip_features' \
        --asr_dir './custom_pipeline/ASR' \
        --asr_feature_dir './custom_pipeline/ASR_feats_all-MiniLM-L6-v2' \
        --eval_batch_size 1 \
        --task_moment_retrieval \
        --task_moment_segmentation \
        --task_step_captioning \
        --ckpt_dir './checkpoints/hirest_joint_model/' \
        --end_to_end

Namespace(train=False, seed=2222, comment='', device='cuda', data_dir='./custom_pipeline/splits/', video_feature_dir='./custom_pipeline/eva_clip_features', asr_dir='./custom_pipeline/ASR', asr_feature_dir='./custom_pipeline/ASR_feats_all-MiniLM-L6-v2', task_moment_retrieval=True, task_moment_segmentation=True, task_step_captioning=True, end_to_end=True, train_batch_size=32, eval_batch_size=1, clip_grad_norm=-1.0, gradient_accumulation_steps=1, epochs=10, optim='adamw', lr=1e-05, fp16=False, weight_decay=0.0, warmup_steps=0.1, n_model_frames=-1, num_workers=4, distributed=False, load=None, ckpt_dir='./checkpoints/hirest_joint_model/', num_thread_reader=16, n_display=16, max_frames_step_captioning=20, max_words=48, visual_num_hidden_layers=2, decoder_num_hidden_layers=2, moment_segmentation_difference_threshold=0.5, moment_segmentation_max_iterations=20, num_beams=5, run_name='clip_g_VR_32frames_avgpool', video_retrieval_model='clip_g', raw_frame=False, save_feats=False, local_rank=0, gp

# Print Final Results

In [7]:
with open("./checkpoints/hirest_joint_model/final_end_to_end_results.json", 'r') as f:
    results = json.load(f)

for prompt in results:
    print("Prompt:", prompt)
    for video in results[prompt]:
        print("Video:", video)

        print("Retrieved Moment:", results[prompt][video]["bounds"])
        
        for step in results[prompt][video]["steps"]:
            print(f"Segment {step['index']}:", step["absolute_bounds"], step["heading"])

FileNotFoundError: [Errno 2] No such file or directory: './checkpoints/hirest_joint_model/final_end_to_end_results.json'

# Compare with original video

In [ ]:
%%html
<iframe width="560" height="315" src="https://www.youtube.com/embed/4Gx9W0XFAkA" title="YouTube video player" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture; web-share" allowfullscreen></iframe>